In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
import json

# 검색 주소(동) 엑셀 파일 찾기

In [12]:
def find_data(query):
    url = 'https://www.code.go.kr/stdcode/regCodeL.do'
    params = {
        'locataddNm': query
    }

    resp = requests.get(url, params=params, verify=False)
    print(resp.request.url)
    soup = BeautifulSoup(resp.text)

    address_list = []
    code = soup.select('td.table_left')
    address = soup.select('td.table_center01')
    address_count = len(address)

    for i in range(address_count):
        address_list.append({
            'code': code[i+1].text,
            'address': address[i].text
        })

    if address_count == 0:
        print("존재하지 않는 주소입니다.")
        return

    elif address_count > 1:
        print("두 개 이상의 결과가 존재합니다.")
        for i in range(address_count):
            print("{}. {}".format(i+1, address_list[i]['address']))
        num = int(input("원하시는 주소의 번호를 입력해주세요. "))
        address_code = address_list[i-1]['code']

    else:
        address_code = address_list[0]['code']

    city = address_code[0:2]
    county = address_code[2:5]
    town = address_code[5:8]

    base_url = 'https://www.juso.go.kr/info/RoadNameDataList.do'
    params_url = '?type=excel&city1={}&county1={}&town1={}'.format(city, county, town)
    file_url = base_url + params_url

    resp = urllib.request.urlopen(file_url)
    data = resp.read()
    df = pd.read_excel(data, encoding='cp949', header=1)

    return df

In [ ]:
find_data("상도동")

C:\Users\D.O\anaconda3\envs\study1\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.code.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\D.O\anaconda3\envs\study1\lib\site-packages\bs4\__init__.py:177: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 194 of the file C:\Users\D.O\anaconda3\envs\study1\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  warnings.warn(self.NO_PARSER_SPECIFIED_WARNING % dict(


https://www.code.go.kr/stdcode/regCodeL.do?locataddNm=%EC%83%81%EB%8F%84%EB%8F%99
두 개 이상의 결과가 존재합니다.
1. 서울특별시 동작구 상도동
2. 경상북도 포항시 남구 상도동


# 카페 데이터 크롤링

In [3]:
def find_cafe_list(df):    
    address_list = []
    umd = df.iloc[0, -1]    # 읍면동 이름 (query값과 동일함) -> 한번 더 비교를 위해 추가함
    
    for i, row in df.iterrows():
        address_list.append(df.at[i, '시도명'] + " "
                            + df.at[i, '시군구명'] + " "
                            +df.at[i, '도로명'])
    
    naver_map_url = "https://map.naver.com/v5/api/search"
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
    }
    result_list = []

    for address in address_list:    # 도로명 Loop
        query = address + " 카페"

        params = {
            'query': query,
            'page': 1,
            'displayCount': 300,    # total_count 입력하여 한 페이지에 모든 정보 출력
            'isPlaceRecommendationReplace': 'true',
            'lang': 'ko'
        }

        resp = requests.get(naver_map_url, params=params, headers=headers)
        resp.encoding=None

        data = json.loads(resp.text)
        result = data.get('result')
        
        if not result['place']:    # 검색결과가 존재하지 않는 경우
            continue

        for place in result['place']['list']:
            # 데이터 저장
            if (('카페' in place['category'] or '카페,디저트' in place['category'] or '테이크아웃커피' in place['category']) # 해당 카테고리만 저장
                and (place['address'].find(umd[:-1]) != -1)):    # 입력한 동 이름과 일치하는지 2차 확인
                menu_list = []
                menu_info = None
                if place['menuInfo']:    # 메뉴 정보가 존재하는 경우
                    menu_list = place['menuInfo'].split("|")
                    menu_name_list = []
                    menu_price_list = []

                    for menu in menu_list:
                        menu_name = ""
                        menu_name_temp = menu.strip().split(" ")[:-1]

                        # 메뉴 이름
                        for idx, word in enumerate(menu_name_temp):
                            menu_name += str(word)
                            if idx != len(menu_name_temp[:-1]):
                                menu_name += " "
                        menu_name_list.append(menu_name)

                        # 메뉴 가격
                        menu_price = menu.strip().split(" ")[-1]
                        menu_price_list.append(menu_price)

                    menu_info = dict(zip(menu_name_list, menu_price_list))

                result_list.append({
                    'name': place['name'],    # 이름이 같은 경우 존재??
                    'tel': place['tel'],
                    'category': place['category'],
                    'address': place['address'],
                    'road_address': place['roadAddress'],
                    'review_count': place['reviewCount'],
                    'homepage': place['homePage'],
                    'menu_info': menu_info
                })
                    
    return result_list

# 최저가 카페 찾기

In [4]:
def find_cheapest(cafe_list):
    final_min_price = None
    result_list = []

    for cafe in cafe_list:
        menu = cafe['menu_info']
        if not menu:
            continue
            
        for name in menu.keys():
            min_price = None
            price_is_int = True
            if name.find('아메리카노') != -1:  #아메리카노라는 문자열이 있는지 확인
                price_str = menu[name]
                price_str = price_str.replace(',','')
                try:
                    price = int(price_str)
                except:
                    continue

                if not price_is_int:
                    continue
                price = int(price_str)

                # 최저가와 비교
                if not min_price:
                    min_price = price
                else:
                    if price < min_price:
                        min_price = price

            if min_price:
                if not final_min_price:
                    final_min_price = min_price
                    result_list.append(cafe)
                else:
                    if min_price < final_min_price:
                        result_list = []
                        result_list.append(cafe)
                        final_min_price = min_price
                    elif min_price == final_min_price:
                        result_list.append(cafe)
    return result_list, final_min_price

# 리뷰 가장 많은 카페 찾기


In [5]:
def find_max_review(cafe_list):
    result_list = []
    max_review = -1
    
    for cafe in cafe_list:
        if cafe['review_count'] > max_review:
            result_list = []
            result_list.append(cafe)
            max_review = cafe['review_count']
        elif cafe['review_count'] == max_review:
            result_list.append(cafe)
            
    return result_list

# Run

In [6]:
def integrated_mod():
    result_list = []
    query = input("동 이름을 입력하세요 (ex. 가산동): ")
    if (query.find(' ') != -1) or (query[-1] != "동"):
        print("잘못된 입력입니다.")
        return
    
    df = find_data(query)
    
    if df is None:    # 입력한 주소가 존재X
        return
    
    cafe_list = find_cafe_list(df)
    
    option = int(input("원하는 검색 조건을 입력하세요.(0.종료, 1.최저가, 2.최다리뷰): "))
    if option == 0:
        return
    
    elif option == 1:
        result_list, min_price = find_cheapest(cafe_list)
        for result in result_list:
            print(result['name'])
            print("주소: " + result['address'])
            print("아메리카노: {}원".format(min_price))
            print("-"*30)
        return result_list
        
    elif option == 2:
        result_list = find_max_review(cafe_list)
        for result in result_list:
            print(result['name'])
            print("주소: " + result['address'])
            print("리뷰: {}".format(result['review_count']))
            print("-"*30)
        return result_list
    
    else:
        print("잘못된 입력입니다.")
        return

In [7]:
run = integrated_mod()

동 이름을 입력하세요 (ex. 가산동): 삼성동


C:\Users\D.O\anaconda3\envs\study1\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.code.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\D.O\anaconda3\envs\study1\lib\site-packages\bs4\__init__.py:177: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 194 of the file C:\Users\D.O\anaconda3\envs\study1\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  warnings.warn(self.NO_PARSER_SPECIFIED_WARNING % dict(


두 개 이상의 결과가 존재합니다.
1. 서울특별시 강남구 삼성동
2. 대전광역시 동구 삼성동
원하시는 주소의 번호를 입력해주세요. 1
원하는 검색 조건을 입력하세요.(0.종료, 1.최저가, 2.최다리뷰): 1
커피온리 봉은사역점
주소: 서울특별시 강남구 삼성동 161-17
아메리카노: 900원
------------------------------


In [21]:
run2 = integrated_mod()

동 이름을 입력하세요 (ex. 가산동): 가산동


C:\Users\user\.conda\envs\study1\lib\site-packages\urllib3\connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.code.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


원하는 검색 조건을 입력하세요.(0.종료, 1.최저가, 2.최다리뷰): 1
매머드커피 에이스가산타워점
주소: 서울특별시 금천구 가산동 550-9 103호
아메리카노: 900원
------------------------------
매머드익스프레스 가산더스카이밸리1차점
주소: 서울특별시 금천구 가산동 371-57
아메리카노: 900원
------------------------------


In [29]:
run2 = integrated_mod()

동 이름을 입력하세요 (ex. 가산동): 가산동


C:\Users\user\.conda\envs\study1\lib\site-packages\urllib3\connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.code.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


원하는 검색 조건을 입력하세요.(0.종료, 1.최저가, 2.최다리뷰): 1
매머드커피 에이스가산타워점
주소: 서울특별시 금천구 가산동 550-9 103호
아메리카노: 900원
------------------------------
매머드익스프레스 가산더스카이밸리1차점
주소: 서울특별시 금천구 가산동 371-57
아메리카노: 900원
------------------------------
